In [ ]:
import os
# print(os.getcwd())
# os.chdir("prachi/saia-finetuning")
print(os.getcwd())
import sys
from argparse import ArgumentParser
import json
from pathlib import Path
from tooling.huggingface_latent_representations.transformers.attention_featurization import extract_latent_feature_graph, extract_fine_grained_latent_feature_graph
 
from torch import arange, device
from torch.cuda import empty_cache
from tqdm import tqdm
from tooling.agents.data import load_csv_file, load_json_file
from time import time
from pandas import DataFrame
import pickle

In [ ]:
import sys

from torch import device
from tqdm import tqdm
from tooling.agents.data import load_csv_file, load_json_file
from time import time
from pandas import DataFrame
import pickle
from argparse import ArgumentParser
import json
from pathlib import Path
hf_token=''

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd


model_name = "meta-llama/Llama-3.1-8B-Instruct" # Replace with "llama31_8b" or your specific model
#model_name = "meta-llama/llama31_8b"
tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, output_attentions=True, torch_dtype=torch.float16, device_map="auto",\
                                             use_auth_token=hf_token)



In [ ]:
def find_subarray( large_list, small_list):
        large_len = len(large_list)
        small_len = len(small_list)
        
        for i in range(large_len - small_len + 1):
            if large_list[i:i+small_len] == small_list:
                return i
        
        return -1

def search_for_target_sequence(tokenizer, target_text, prompt, input_ids):
    '''
    grab the template part from input
    '''
    tokenized_text = tokenizer.tokenize(target_text)
    reference_tokens = tokenizer.tokenize(prompt)
    # print(tokenized_text)
    # print(reference_tokens)
    # assert len(reference_tokens) == input_ids.shape[1] -1 # this model will automatically add one token at the beginning
    assert set(tokenized_text).issubset(set(reference_tokens)), "target tokens are not subset of prompt"
    last_index = find_subarray(reference_tokens, tokenized_text)
    assert last_index != -1, "token not found"
    # last_index = reference_tokens.index(tokenized_text[-1]) # find the position of last token in target text
    return last_index + len(tokenized_text) - 1

# def search_for_target_sequence(tokenizer,target_text, prompt, input_ids):
#         '''
#         grab the template part from input
#         '''
#     #     messages = [
#     #     {'role': 'user', 'content': prompt},
#     # ]
#         reference_tokens = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, return_tensors="pt").to(model.device)
#     #     messages = [
#     #     {'role': 'user', 'content': target_text},
#     # ]
#         tokenized_text = tokenizer.apply_chat_template(target_text, add_generation_prompt=True, return_tensors="pt").to(model.device)
#         print("in",tokenized_text)
#         print("ref",reference_tokens)
    
#         tokenized_text1=np.array(tokenized_text[0][7:-5].cpu())
#         reference_tokens1=np.array(reference_tokens[0][7:-5].cpu())
        
#         print("in",tokenized_text1)
#         print("ref",reference_tokens1)
#         # assert len(reference_tokens) == input_ids.shape[1] -1 # this model will automatically add one token at the beginning
#         # assert set(tokenized_text).issubset(set(reference_tokens))
#         last_index = find_subarray(reference_tokens1, tokenized_text1)
#         assert last_index != -1, "token not found"
#         # last_index = reference_tokens.index(tokenized_text[-1]) # find the position of last token in target text
#         return last_index + len(tokenized_text) - 1

def llama_extract_latent_features(messages, max_tokens):
    #prompt = generate_llama31_input(orch_prompt, str(ast.literal_eval(messages)))

    #prompt = generate_llama31_input(orch_prompt, messages)
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

    # inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        inputs,
        max_new_tokens=max_tokens,  # Adjust max_length as needed
        return_dict_in_generate=True,
        output_hidden_states=True,
        output_attentions=True,
        temperature=0.01,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
        #output_scores=True
    )

    # Decode the generated text
    #response = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
    
    return inputs[0], output[0]




def find_keywords(tokenizer, keywords, prompt, input_ids):
    tokenized_text = tokenizer.tokenize(keywords) # for matching purpose excluding the start and end token of provided matching tokens
    all_tokens = tokenizer.tokenize(prompt)
    indices = []
    i = 0
    while i <= len(all_tokens) - len(tokenized_text):
        if all_tokens[i:i+len(tokenized_text)] == tokenized_text:
            indices.append(i)
            i += len(tokenized_text)
        else:
            i += 1
    return indices

def get_llama_out_wattention(input1):
    prompt = generate_llama31_input(orch_prompt, input1)

    #prompt = generate_llama31_input(orch_prompt, messages)

    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        max_new_tokens=100,  # Adjust max_length as needed
        return_dict_in_generate=True,

        output_attentions=True,
        output_scores=True
    )

    # Decode the generated text
    response = tokenizer.decode(output.sequences[0], skip_special_tokens=True)



def construct_graph(
    input_ids,
    outputs,
    function_name_start_ids, 
    function_name_end_ids, 
    template_end_ids, 
    input_end_ids,
    num_function
):
    #  get all the input ids for input template
    tokenids = []
    for i in range(num_function):
        each_function = (function_name_start_ids[i],function_name_end_ids[i]) # each pair of start and end positions
        # each_argument = (function_argument_start_ids[i],function_argument_end_ids[i])

        tokenids.append(arange(each_function[0], each_function[1]))
        # tokenids.append(arange(each_argument[0], each_argument[1]))
    
    if input_end_ids == template_end_ids + 1: # this is for the shortest user question, when there is only one token
        tokenids.append(arange(template_end_ids, input_end_ids))
    else:
        tokenids.append(arange(template_end_ids, input_end_ids))
    print(len(tokenids), (num_function * 2 + 1))
    # assert len(tokenids) == num_function * 2 + 1

    # all_functions is a list of list for input ids
    return extract_fine_grained_latent_feature_graph(
        input_ids=input_ids,
        outputs=outputs,
        target_ids=tokenids
    )


def get_function_token_offset(
    tokenizer,
    input_ids,
    prompt:str,
    num_function:int,
    user_question_start_str:str="<|eot_id|><|start_header_id|>user<|end_header_id|>",
    assistant_start_str:str="<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
):
    function_name_start_ids = find_keywords(tokenizer, keywords='name', prompt=prompt, input_ids=input_ids)
    function_name_end_ids = find_keywords(tokenizer, keywords='required', prompt=prompt, input_ids=input_ids)
    #find_keywords(tokenizer, keywords='"parameters": {"type": "object", ', prompt=prompt, input_ids=input_ids)
    # function_argument_start_ids = find_keywords(tokenizer, keywords='"properties": {"', prompt=prompt, input_ids=input_ids)
    # function_argument_end_ids = find_keywords(tokenizer, keywords=', "required": [', prompt=prompt, input_ids=input_ids)

    # assert len(function_name_start_ids) == len(function_name_end_ids) 
    #== len(function_argument_start_ids) == len(function_argument_end_ids) == num_function
    try:
        template_end_ids = search_for_target_sequence(tokenizer, user_question_start_str, prompt, input_ids)
    except:
        template_end_ids = search_for_target_sequence(tokenizer, user_question_start_str[:-2], prompt, input_ids) # match for 
    input_end_ids = search_for_target_sequence(tokenizer, assistant_start_str, prompt, input_ids)
    
    return function_name_start_ids, function_name_end_ids, template_end_ids, input_end_ids

    

In [5]:
import pickle
with open("data_paper.pkl","rb") as f:
    data_samples_edited=pickle.load(f)
# use top 100 valid samples from data_samples_edited
data_input=[]
tool_names=["sentiment_classification","same_or_different","news_summarization","question_answer","human_reasoning"]

for i in range(len(tool_names)):
    start=i*500
    print(i)
    print(start,start+100)
    data_input.extend(data_samples_edited[start:start+100])
    print(len(data_input))
print(len(data_input))
del data_samples_edited

0
0 100
100
1
500 600
200
2
1000 1100
300
3
1500 1600
400
4
2000 2100
500
500


In [9]:
# print(len(data_samples_edited))

In [6]:
# set dynamic prompt based on tools available
def out_sys_prompt(tools, tool_names):
    return f"""
    You are an expert in picking functions. You are given a user input and a set of possible functions.
    Based on the user input, you will need to pick single function to achieve the purpose you infer from the input.
    You do not provide any explanation, additional text, or answers to user's input as your output. 
    A STRICT RULE, do not answer any of the user questions and do not comment on the input. STRICTLY only output the function for the user's query from this list of functions:{tool_names}"
    STRICTLY follow this output format for your response:
    Example query: I love this song, this was the best song I have heard in a long time.
    Your output: sentiment_classification
    Strictly only output the picked function name without any additional text.
    Here is a list of functions in JSON format that you can invoke. 
    {tools}\n
    """


In [7]:
tool_names=["sentiment_classification","same_or_different","news_summarization","question_answer","human_reasoning"]
tools = [
    {
        "name": "sentiment_classification",
        "description": "Sentiment classifier takes in a user review input and classifies it into a positive or negative sentiment. User query will tend to have either positive or negative things to say about the product.",
        "examples": ["This is a text book I ordered for my classes for teacher certification. This book is very easy to read, and I am really enjoying it. It presents info and then at the end of a section, recaptures the essence of the reading for you. I would suggest it for any teaching class. I am learning so much from this book. I have been buying all my text books from Amazon because they are so much cheaper, and sometimes I can get free shipping. So far, there has not been a book that I could not find here at Amazon. "],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User input reviewing a product. User sentiment about the product will be either positive or negative."
                },
                "output": {
                    "type": "string",
                    "description": "Positive or Negative classification of the user sentiment."
                }
            },
            "required": []
        }
    },
        {
        "name": "news_summarization",
        "description": "News Summarizer skill is built for summarizing or in other words explaining the meaning of the news input given by the user. Input topics may vary accross sports, politics, news from events and countries however input will be a news story.",
        "examples": ["""When your grandmother is one of the most famous cosmetics moguls in history, it might put a little pressure on you to succeed. But for Aerin Lauder, the 44-year-old granddaughter of Estée Lauder, 
        who founded the eponymous make-up company, the legacy has been an inspiration rather than a burden. 
        Lauder worked her way up through the ranks of the billion-dollar family company for 25 years to the position of style and image director. 
        In 2012, she decided to combine her passion for home décor with her knowledge of beauty, to launch her own lifestyle brand, called AERIN. 
        The businesswoman spoke to CNN's Kirstie Lu Stout about her drive, the importance of saying "no," and how beauty transcends borders or race. 
        CNN: Are you living out your dream today with your own brand, AERIN? Aerin Lauder: I'm definitely living my dream. 
        As a little girl, I've always loved beauty and I loved home, so I've managed to combine the two into a brand. 
        CNN: What is it like to live up to the heritage of Estee Lauder?"""],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query asking a summary of the incident. User query may have the word summary or summarize or findings in it."
                },
                "output": {
                    "type": "string",
                    "description": "A natural language summary of the news incident."
                }
            },
            "required": []
        }
    },
        {
        "name": "same_or_different",
        "description": "Same or Different classifier takes in two input sentences or questions from the user and classifies them to either have same meaning or different meaning.",
        "examples": ["All I care about is money, how much more money will I save if I don't have any kids? and Does ""an eye for an eye"" apply in legal situations?"" "],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User input will have two sentences or questions. These sentences may have same meaning or different"
                },
                "output": {
                    "type": "string",
                    "description": "Same or Different classification of the user input."
                }
            },
            "required": []
        },
        },
            
                {
        "name": "human_reasoning",
        "description": "Given an input problem statement, multiple solution options and the correct option picked by a human, the human reasoning module will give a logic or reason as to why that option is correct.",
        "examples": ["""Question: If one wanted great bar service and maybe a room afterwards, where would be a good place to go? Options:
- entertainment
- train
- motel
- hotel
- wedding
Explain why a human would choose hotel to answer the question above: """],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "A problem statement with multiple answer options and a correct option picked by human."
                },
                "output": {
                    "type": "string",
                    "description": "A reasoning/logic for picking the correct option"
                }
            },
            "required": []
        },
                },
                                    {
        "name": "question_answer",
        "description": "Given a passage and a question, the question_answer module will answer the question based on the information in the input passage.",
        "examples": ["""Airborne Interception radar (AI) was unreliable. The heavy fighting in the Battle of Britain had eaten up most of Fighter Command's resources, so there was little investment in night fighting. Bombers were flown with airborne search lights out of desperation[citation needed], but to little avail. Of greater potential was the GL (Gunlaying) radar and searchlights with fighter direction from RAF fighter control rooms to begin a GCI system (Ground Control-led Interception) under Group-level control (No. 10 Group RAF, No. 11 Group RAF and No. 12 Group RAF).",
What was an issue with not having AI?"""],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "A passage followed by a question whose answer is in the passage."
                },
                "output": {
                    "type": "string",
                    "description": "Answer to the question in user input."
                }
            },
            "required": []
        },
                                    }
        ]
orch_prompt=out_sys_prompt(tools, tool_names)  

In [19]:
# call model
import numpy as np
import torch.nn.functional as F

# inputs = tokenizer(prompt, return_tensors="pt")
# output = model.generate(
#     **inputs,
#     max_new_tokens=100,  # Adjust max_length as needed
#     return_dict_in_generate=True,
#     output_hidden_states=True,
#     output_attentions=True,
#     #output_scores=True
# )

# sequences=output.sequences
# response=[tokenizer.decode(seq, skip_special_tokens=True) for seq in sequences]
# print(f"Generated Response: {response}")




query=data_input[1]["input"]
# prompt = generate_llama31_input(orch_prompt, query)
# inputs = tokenizer(prompt, return_tensors="pt")
messages = [
    {'role': 'system', 'content': orch_prompt},
    {'role': 'user', 'content': query}
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False)+"<|start_header_id|>assistant<|end_header_id|>"
# print(prompt)
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
inputs={"input_ids":inputs}
with torch.no_grad():
    outputs=model(**inputs)
    logits=outputs.logits
probs=F.softmax(logits,dim=1)
print(probs)
print(probs.shape)
    
# # outputs = model.generate(inputs, max_new_tokens=1, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
# print("outpp",tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

# print(inputs)

# set_a_start_ids = search_for_target_sequence(tokenizer,"required", prompt, inputs)

# print(set_a_start_ids)

# set_a_start_ids = search_for_target_sequence(tokenizer,"required", prompt, inputs)

# print(set_a_start_ids)

# # input_ids= tokenizer(prompt, return_tensors="pt")
# tmp1=np.array(np.array(inputs[0].cpu())[set_a_start_ids])
# # tmp1
# print(tokenizer.decode(tmp1))


tensor([[[0.0000e+00, 0.0000e+00, 2.0802e-04,  ..., 4.4525e-05,
          4.4525e-05, 4.4525e-05],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.3555e-04,
          9.3555e-04, 9.3555e-04],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 7.1192e-04,
          7.1192e-04, 7.1192e-04],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.3630e-02,
          3.3630e-02, 3.3630e-02],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.7824e-03,
          1.7824e-03, 1.7824e-03],
         [3.8743e-06, 1.4186e-04, 2.4567e-03,  ..., 1.0896e-04,
          1.0896e-04, 1.0896e-04]]], device='cuda:0', dtype=torch.float16)
torch.Size([1, 1558, 128256])


In [20]:
import torch.nn.functional as F

import numpy as np
label_map_file_path="grading/data/label_mapping.json"
max_tokens=1
responses = []
graph_latency = []
response_latency = []
attention_graphs = []
all_labels=[]
output_probs=[]
output_idx=[]
# output_hidden=output_hidden.to(device("cpu")
# refactor the code to have two columns, one for examples, one for reference skills
for idx in range(len(data_input)):
    print(idx)
    label=data_input[idx]['idx']
# for idx, (each_exp, correct_skill, cot1, cot2) in enumerate(data.values.tolist()): for RAG+CoT
    print(data_input[idx]['input'])
    print(tool_names[label])
    all_labels.append(tool_names[label])
    
    # prompt = generate_llama31_input(orch_prompt, input_data[idx])
    start = time()
    messages = [
    {'role': 'system', 'content':orch_prompt},
    {'role': 'user', 'content': data_input[idx]['input']}
    ]
    # prompt = tokenizer.apply_chat_template(messages, tokenize=False)+"<|start_header_id|>assistant<|end_header_id|>"
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    # inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    inputs={"input_ids":inputs}
    with torch.no_grad():
        outputs=model(**inputs)
        logits=outputs.logits
    probs=F.softmax(logits,dim=-1) # softmax over vocab
    probs=probs[0].to("cpu").float() # remove batch dim
    topk=50
    top_p,top_i=torch.topk(probs,topk,dim=-1)

    # outputs1 = model.generate(inputs, max_new_tokens=max_tokens, do_sample=False,output_hidden_states=True,output_attentions=False, return_dict_in_generate=True,eos_token_id=tokenizer.eos_token_id)
    output_probs.append(top_p)
    output_idx.append(top_i)
    del probs

    # response=(tokenizer.decode(outputs1.sequences[0], skip_special_tokens=True))

    # response=str(response)
    # id1=(response).find("assistant")
    # print(response[id1:])
    
    # responses.append(response[id1:])
    # end=time()
    # response_latency.append(end-start)
    # del outputs1
    empty_cache()
            # first get start and end ids and then construct graph
    # function_name_start_ids, function_name_end_ids, template_end_ids, input_end_ids = get_function_token_offset(
    #     tokenizer,
    #     inputs[0],
    #     prompt,
    #     num_function=5,
    #     user_question_start_str="<|eot_id|><|start_header_id|>user<|end_header_id|>",
    #     assistant_start_str="<|start_header_id|>assistant<|end_header_id|>",
    # )
    #     # print(function_name_start_ids, function_name_end_ids, template_end_ids, input_end_ids)
    # tmp=arange(template_end_ids+1, input_end_ids-3)
    # tmp1=np.array(np.array(inputs[0].cpu())[tmp])
    # print("query ",tokenizer.decode(tmp1))
    # print(inputs)
    #     # Build a graph with a graph contraction grouping each function and user token ids
    # attention_graph = construct_graph(
    #     inputs[0],
    #     outputs1,
    #     function_name_start_ids,
    #     function_name_end_ids,
    #     template_end_ids+1,
    #     input_end_ids-3,
    #     num_function=5
    # )
    #     # print(attention_graph)
    # attention_graph.y = label
    # attention_graph.detach().to(device("cpu"))
    # attention_graphs.append(attention_graph)
    # empty_cache()
    # end = time()
    # graph_latency.append(end-start)
    

    

0
This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
sentiment_classification
1
I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.
sentiment_classification
2
This soundtrack is my favorite music of all time, hands down. The intense sadn

In [ ]:
output_directory="grading/data/llama/paper"
torch.save({
    "top_p":output_probs,
    "top_i":output_idx
},Path(output_directory, "output_probs_5class.pt"))

In [17]:
# save responses and attention graphs
import pickle
output_directory="grading/data/llama/paper"
# df.to_csv(Path(output_directory, "responses11.csv"), index=False, header=True)

with open(Path(output_directory, "label_mapping_maxtok1.pkl"), "wb") as f:
    pickle.dump(label_mapping, f)
with open(Path(output_directory, "responses_maxtok1.pkl"), "wb") as f:
    pickle.dump(responses, f)

with open(Path(output_directory, "graph_latency_maxtok1.pkl"), "wb") as f:
    pickle.dump(graph_latency, f)
with open(Path(output_directory, "labels_maxtok1.pkl"), "wb") as f:
    pickle.dump(all_labels, f)
with open(Path(output_directory, "attention_graph_maxtok1.pkl"), "wb") as f:
    pickle.dump(attention_graphs, f)
# DataFrame({

#         "responses": responses,

#     }).to_csv(Path(output_directory, "responses1.csv"), index=False, header=True)

In [21]:
output_directory="grading/data/llama/paper"

with open(Path(output_directory, "output_logits_5class.pkl"), "wb") as f:
    pickle.dump(output_logits, f)
# with open(Path(output_directory, "labels_5classdataset.pkl"), "wb") as f:
#     pickle.dump(all_labels, f)
# with open(Path(output_directory, "responses_5classdataset.pkl"), "wb") as f:
#     pickle.dump(responses, f)

OSError: [Errno 28] No space left on device

In [25]:
# get avg latency and acc
print(np.mean(np.array(graph_latency)))
def get_acc(list1,list2):
    matches=0
    for a,b in zip(list1,list2):
        b_clean=b.replace("assistant","").strip()
        if a==b_clean:
            matches+=1
        else:
            print(a,b_clean)
    return matches/len(list1)
print(get_acc(all_labels,responses))


1.3219775061607362
sentiment_classification question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
human_reasoning
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
question_answer
same_or_different question_answer
question_answer
same_or_different question_answer
question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different question_answer
same_or_different human_reasoning
question_answer
same_or_different sentiment_classification
question_answer
sam

In [26]:
x1=attention_graphs[0]
print(x1)

Data(x=[6, 4096], edge_index=[2, 6], edge_attr=[6, 32], node_types=[6], y=0)


In [27]:
print(x1.edge_index)

tensor([[0, 1, 2, 3, 4, 5],
        [5, 5, 5, 5, 5, 5]])


In [28]:
print(x1.edge_attr)

tensor([[6.7688e-02, 1.3115e-02, 4.1077e-02, 7.5264e-03, 2.7409e-03, 3.8872e-03,
         3.7964e-02, 1.4359e-02, 1.5459e-03, 1.7900e-03, 3.2101e-03, 2.9388e-02,
         5.5122e-04, 3.1281e-03, 1.3387e-04, 7.2670e-04, 3.6469e-03, 5.6458e-03,
         1.4563e-03, 1.2428e-02, 8.9417e-03, 1.6998e-02, 1.5350e-02, 2.3937e-03,
         3.0880e-03, 7.0143e-04, 1.4221e-02, 1.3752e-03, 1.1078e-02, 1.6373e-02,
         1.4481e-02, 3.0537e-03],
        [1.0811e-02, 6.1378e-03, 3.9101e-03, 3.9005e-03, 6.6471e-04, 2.3117e-03,
         1.1520e-02, 1.8738e-02, 8.7309e-04, 1.1702e-03, 2.1629e-03, 7.9880e-03,
         7.4244e-04, 3.8624e-03, 1.0147e-03, 1.2159e-03, 1.5144e-02, 2.3174e-03,
         1.0319e-03, 4.7150e-02, 7.2441e-03, 4.7058e-02, 1.6983e-02, 5.2643e-03,
         1.7881e-03, 1.3180e-03, 4.0283e-03, 8.6899e-03, 1.7410e-02, 5.2948e-03,
         3.6530e-02, 1.0042e-03],
        [1.8799e-02, 1.0735e-02, 4.0588e-03, 5.1003e-03, 2.0332e-03, 7.4673e-04,
         1.2436e-02, 7.4585e-02, 3.3035e-

In [29]:
print(x1.node_types)

tensor([0, 1, 2, 3, 4, 5])
